In [1]:
import pandas as pd
import datetime
import calendar
import holidays
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe' # or 'notebook' or 'colab' or 'jupyterlab'

In [2]:
print(calendar.calendar(2022))

                                  2022

      January                   February                   March
Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su
                1  2          1  2  3  4  5  6          1  2  3  4  5  6
 3  4  5  6  7  8  9       7  8  9 10 11 12 13       7  8  9 10 11 12 13
10 11 12 13 14 15 16      14 15 16 17 18 19 20      14 15 16 17 18 19 20
17 18 19 20 21 22 23      21 22 23 24 25 26 27      21 22 23 24 25 26 27
24 25 26 27 28 29 30      28                        28 29 30 31
31

       April                      May                       June
Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su
             1  2  3                         1             1  2  3  4  5
 4  5  6  7  8  9 10       2  3  4  5  6  7  8       6  7  8  9 10 11 12
11 12 13 14 15 16 17       9 10 11 12 13 14 15      13 14 15 16 17 18 19
18 19 20 21 22 23 24      16 17 18 19 20 21 22      20 21 22 23 24 25 26
25 26 27 28 29 30         23 24 

In [3]:
# 변수 정리
PRODUCT_NAME = "320kV PMJ"
START_TIME = datetime.datetime(2022, 9, 5, 9, 0)
INJECTION_TIME = datetime.timedelta(hours=2)             # 사출 소요 시간
CURING_TIME = datetime.timedelta(hours=19.5)               # 경화 소요 시간 
COOLING_TIME = datetime.timedelta(hours=12)              # 냉각 소요 시간
MOLD_RESET_TIME = datetime.timedelta(hours=3)            # 금형 탈형 및 조립 소요 시간
MOLD_PREHEATING_TIME = datetime.timedelta(hours=4)       # 금형 예열 소요 시간
DRYING_TIME = datetime.timedelta(days=10)                # 접속재 건조 소요 시간
OUTSOURCING_TIME = datetime.timedelta(days=3)            # 외주 가공 소요 시간
FINAL_TOUCH_TIME = datetime.timedelta(days=3)        # 삼중점 그라인딩 + 차폐몰딩 + 세척/포장 등
WORKING_TIME_MIN = 9  # 오전 9시
WORKING_TIME_MAX = 16  # 오후 4시
PROCESS_NAMES = ["사출", "경화", "냉각", "탈형/조립", "건조", "외주가공", "마무리(그라인딩, 차폐몰딩 등, 세척/포장)"]

In [4]:
def check_if_working_day(date_time, 
                         work_on_saturday=False, work_on_sunday=False,
                         work_on_holiday=False, three_day_shift=False):
    """
    작업일인지 확인
    
    Parameters
    ----------
    date_time         : datetime (datetime.datetime(2022, 8, 18, 9, 0))
                        날짜
                    
    work_on_saturday  : boolean
                        토요일 근무 여부
    
    work_on_sunday    : boolean
                        일요일 근무 여부
    
    work_on_holiday   : boolean
                        공휴일 근무 여부
    
    three_day_shift   : boolean
                        잠 안자고 네버스탑???
                        (this option overwrite other parameters)

    Returns
    -------
    작업일 여부 : boolean

    Examples
    --------
    >>> check_if_working_day(datetime.datetime(2022, 8, 18, 9, 0),
            work_on_saturday=False, work_on_sunday=False,
            work_on_holiday=False, three_day_shift=False)
    """
    if three_day_shift == True:
        return True
    else:
        if work_on_saturday == True and work_on_sunday == True and work_on_holiday == True:
            if WORKING_TIME_MIN <= date_time.hour <= WORKING_TIME_MAX:
                return True
            else:
                return False
        elif work_on_saturday == True and work_on_sunday == True and work_on_holiday == False:
            if date_time not in holidays.KR():
                if WORKING_TIME_MIN <= date_time.hour <= WORKING_TIME_MAX:
                    return True
                else:
                    return False
            else:
                return False
        elif work_on_saturday == True and work_on_sunday == False and work_on_holiday ==True:
            if date_time.weekday() < 6:
                if WORKING_TIME_MIN <= date_time.hour <= WORKING_TIME_MAX:
                    return True
                else:
                    return False
            else:
                return False
        elif work_on_saturday == True and work_on_sunday == False and work_on_holiday == False:
            if date_time.weekday() < 6 and date_time not in holidays.KR():
                if WORKING_TIME_MIN <= date_time.hour <= WORKING_TIME_MAX:
                    return True
                else:
                    return False
            else:
                return False 
        elif work_on_saturday == False and work_on_sunday == True and work_on_holiday == True:
            if date_time.weekday() < 5 or date_time.weekday() == 6:
                if WORKING_TIME_MIN <= date_time.hour <= WORKING_TIME_MAX:
                    return True
                else:
                    return False
            else:
                return False 
        elif work_on_saturday == False and work_on_sunday == True and work_on_holiday == False:
            if (date_time.weekday() < 5 or date_time.weekday() == 6) and date_time not in holidays.KR():
                if WORKING_TIME_MIN <= date_time.hour <= WORKING_TIME_MAX:
                    return True
                else:
                    return False
            else:
                return False
        elif work_on_saturday == False and work_on_sunday == False and work_on_holiday == True:
            if date_time.weekday() < 5:
                if WORKING_TIME_MIN <= date_time.hour <= WORKING_TIME_MAX:
                    return True
                else:
                    return False
            else:
                return False
        else:
            if date_time.weekday() < 5 and date_time not in holidays.KR():
                if WORKING_TIME_MIN <= date_time.hour <= WORKING_TIME_MAX:
                    return True
                else:
                    return False
            else:
                return False

In [5]:
# Test Conditions

assert check_if_working_day(
    datetime.datetime(2022, 8, 21, 13),  # 토요일, 주말 일하는 조건
    work_on_saturday=True, work_on_sunday=True,
    work_on_holiday=False, three_day_shift=False
) == True, "Something is wrong_1"

assert check_if_working_day(
    datetime.datetime(2022, 8, 21, 13),  # 토요일 오후 1시
    work_on_saturday=False, work_on_sunday=False,
    work_on_holiday=False, three_day_shift=False
) == False, "Something is wrong_2"

assert check_if_working_day(
    datetime.datetime(2022, 8, 18, 8),  # 평일 오전 8시
    work_on_saturday=False, work_on_sunday=False,
    work_on_holiday=False, three_day_shift=False
) == False, "Something is wrong_3"

assert check_if_working_day(
    datetime.datetime(2022, 8, 18, 9),  # 평일 오전 9시
    work_on_saturday=False, work_on_sunday=False,
    work_on_holiday=False, three_day_shift=False
) == True, "Something is wrong_4"

assert check_if_working_day(
    datetime.datetime(2022, 8, 18, 16),  # 평일 오후 4시
    work_on_saturday=False, work_on_sunday=False,
    work_on_holiday=False, three_day_shift=False
) == True, "Something is wrong_5"

assert check_if_working_day(
    datetime.datetime(2022, 8, 18, 17),  # 평일 오후 5시
    work_on_saturday=False, work_on_sunday=False,
    work_on_holiday=False, three_day_shift=False
) == False, "Something is wrong_6"

assert check_if_working_day(
    datetime.datetime(2022, 9, 10, 10),  # 추석 오전 10시
    work_on_saturday=False, work_on_sunday=False,
    work_on_holiday=False, three_day_shift=False
) == False, "Something is wrong_7"

assert check_if_working_day(
    datetime.datetime(2022, 1, 1, 10),  # 설 오전 10시
    work_on_saturday=False, work_on_sunday=False,
    work_on_holiday=False, three_day_shift=False
) == False, "Something is wrong_8"

assert check_if_working_day(
    datetime.datetime(2022, 1, 1, 10),  # 설(토요일) 오전 10시
    work_on_saturday=True, work_on_sunday=False,
    work_on_holiday=True, three_day_shift=False
) == True, "Something is wrong_9"

In [6]:
def calculate_production_time(
    num_joints, start_time, 
    work_on_saturday=False, work_on_sunday=False, work_on_holiday=False, three_day_shift=False
):
    """
    접속재 제조 소요일 계산
    
    Parameters
    ----------
    num_joints    : int
                    접속재 필요 수량
    
    start_time     : datetime (datetime.datetime(2022, 8, 18, 9, 0))
                     작업 시작 시간

    work_on_saturday  : boolean
                        토요일 근무 여부
    
    work_on_sunday    : boolean
                        일요일 근무 여부
    
    work_on_holiday   : boolean
                        공휴일 근무 여부
    
    three_day_shift   : boolean
                        잠 안자고 네버스탑???
                        (this option overwrite other parameters)

    Returns
    -------
    접속재 제조 소요일 : dataframe

    Examples
    --------
    >>> calculate_production_time(5, datetime.datetime(2022, 8, 18, 9, 0),
            work_on_saturday=False, work_on_sunday=False,
            work_on_holiday=False, three_day_shift=False)
    """
    # Initialization of list for times
    start = []
    finish = []
    
    num_joint = 0
    
    time = start_time
    
    while num_joint < num_joints:
        if check_if_working_day(time, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift):
            start.append(time)
            time = time + INJECTION_TIME
            finish.append(time)
            start.append(time)
            time = time + CURING_TIME
            finish.append(time)
            start.append(time)
            time = time + COOLING_TIME
            finish.append(time)
            
            if check_if_working_day(time, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift):
                start.append(time)
                time = time + MOLD_RESET_TIME
                finish.append(time)
                start.append(time)              # 건조 시간 추가
                time_2 = time + DRYING_TIME     # 건조 시간 추가
                finish.append(time_2)           # 건조 시간 추가
                if check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift):
                    start.append(time_2)                   # 외주 가공 시간 추가
                    time_2 = time_2 + OUTSOURCING_TIME     # 외주 가공 시간 추가
                    time_2 = time_2.replace(hour=10)      # 외주 가공 시간 추가
                    finish.append(time_2)                  # 외주 가공 시간 추가
                    if check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift):
                        start.append(time_2)                    # 마무리 시간 추가
                        time_2 = time_2 + FINAL_TOUCH_TIME      # 마무리 시간 추가
                        finish.append(time_2)                   # 마무리 시간 추가
                        num_joint += 1
                        time = time + MOLD_PREHEATING_TIME
                    else:
                        while check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift) == False:
                            time_2 = time_2 + datetime.timedelta(hours=1)
                        start.append(time_2)                    # 마무리 시간 추가
                        time_2 = time_2 + FINAL_TOUCH_TIME      # 마무리 시간 추가
                        finish.append(time_2)                   # 마무리 시간 추가
                        num_joint += 1
                        time = time + MOLD_PREHEATING_TIME
                else:
                    while check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift) == False:
                        time_2 = time_2 + datetime.timedelta(hours=1)
                    start.append(time_2)                   # 외주 가공 시간 추가
                    time_2 = time_2 + OUTSOURCING_TIME     # 외주 가공 시간 추가
                    time_2 = time_2.replace(hour=10)      # 외주 가공 시간 추가
                    finish.append(time_2)                  # 외주 가공 시간 추가
                    if check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift):
                        start.append(time_2)                    # 마무리 시간 추가
                        time_2 = time_2 + FINAL_TOUCH_TIME      # 마무리 시간 추가
                        finish.append(time_2)                   # 마무리 시간 추가
                        num_joint += 1
                        time = time + MOLD_PREHEATING_TIME
                    else:
                        while check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift) == False:
                            time_2 = time_2 + datetime.timedelta(hours=1)
                        start.append(time_2)                    # 마무리 시간 추가
                        time_2 = time_2 + FINAL_TOUCH_TIME      # 마무리 시간 추가
                        finish.append(time_2)                   # 마무리 시간 추가
                        num_joint += 1
                        time = time + MOLD_PREHEATING_TIME

            else:
                while check_if_working_day(time, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift) == False:
                    time = time + datetime.timedelta(hours=1)
                start.append(time)
                time = time + MOLD_RESET_TIME
                finish.append(time)
                start.append(time)                # 건조 시간 추가
                time_2 = time + DRYING_TIME     # 건조 시간 추가
                finish.append(time_2)           # 건조 시간 추가
                if check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift):
                    start.append(time_2)                   # 외주 가공 시간 추가
                    time_2 = time_2 + OUTSOURCING_TIME     # 외주 가공 시간 추가
                    time_2 = time_2.replace(hour=10)      # 외주 가공 시간 추가
                    finish.append(time_2)                  # 외주 가공 시간 추가
                    if check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift):
                        start.append(time_2)                    # 마무리 시간 추가
                        time_2 = time_2 + FINAL_TOUCH_TIME      # 마무리 시간 추가
                        finish.append(time_2)                   # 마무리 시간 추가
                        num_joint += 1
                        time = time + MOLD_PREHEATING_TIME
                    else:
                        while check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift) == False:
                            time_2 = time_2 + datetime.timedelta(hours=1)
                        start.append(time_2)                    # 마무리 시간 추가
                        time_2 = time_2 + FINAL_TOUCH_TIME      # 마무리 시간 추가
                        finish.append(time_2)                   # 마무리 시간 추가
                        num_joint += 1
                        time = time + MOLD_PREHEATING_TIME
                else:
                    while check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift) == False:
                        time_2 = time_2 + datetime.timedelta(hours=1)
                    start.append(time_2)                   # 외주 가공 시간 추가
                    time_2 = time_2 + OUTSOURCING_TIME     # 외주 가공 시간 추가
                    time_2 = time_2.replace(hour=10)      # 외주 가공 시간 추가
                    finish.append(time_2)                  # 외주 가공 시간 추가
                    if check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift):
                        start.append(time_2)                    # 마무리 시간 추가
                        time_2 = time_2 + FINAL_TOUCH_TIME      # 마무리 시간 추가
                        finish.append(time_2)                   # 마무리 시간 추가
                        num_joint += 1
                        time = time + MOLD_PREHEATING_TIME
                    else:
                        while check_if_working_day(time_2, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift) == False:
                            time_2 = time_2 + datetime.timedelta(hours=1)
                        start.append(time_2)                    # 마무리 시간 추가
                        time_2 = time_2 + FINAL_TOUCH_TIME      # 마무리 시간 추가
                        finish.append(time_2)                   # 마무리 시간 추가
                        num_joint += 1
                        time = time + MOLD_PREHEATING_TIME
            
        else:
            while check_if_working_day(time, work_on_saturday, work_on_sunday, work_on_holiday, three_day_shift) == False:
                time = time + datetime.timedelta(hours=1)
    
    col_names = ["Number", "Process", "Start", "Finish"]
    process_names = [f"{PROCESS_NAMES[j]}_{i+1}" for i in range(num_joints) for j in range(len(PROCESS_NAMES)) ]
    numbers = [str(i+1) for i in range(num_joints) for j in range(len(PROCESS_NAMES))]

    df = pd.DataFrame(index=range(len(process_names)), columns=col_names)
    
    df["Number"] = numbers
    df["Process"] = process_names
    df["Start"] = start
    df["Finish"] = finish

    return df

In [7]:
df = calculate_production_time(
        8, START_TIME, work_on_saturday=False, work_on_sunday=False, work_on_holiday=False, three_day_shift=False
)

In [8]:
fig = px.timeline(df, x_start="Start", x_end="Finish", y="Process", color="Number")
fig.update_yaxes(autorange="reversed", title="공정 순서", title_font_size=20)
fig.update_xaxes(title="시간", title_font_size=20, dtick="d1")
fig.update_layout(autosize=False, height=1500, width=1000, 
                  title=f"{PRODUCT_NAME} 제조 계획", title_font_size=30, title_x=0.5,
                  showlegend=False)
fig.show();

In [9]:
fig.write_html("production_schedule.html")